In [55]:
from azure.identity import AzureCliCredential

credential = AzureCliCredential()
from azure.ai.ml import MLClient
from azure.identity import AzureCliCredential



credential = AzureCliCredential()

ml_client = MLClient(
    credential=credential,
    subscription_id="6fab868f-e7ff-4000-b366-b5d0a27cc581",
    resource_group_name="iat",
    workspace_name="shafi2"
)


In [2]:
pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o


Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:

import os
import pandas as pd
import h2o

def init():
    import logging
    global model

    # Initialize H2O
    h2o.init()

    # Use current directory if AZUREML_MODEL_DIR is not set
    model_dir = os.getenv("AZUREML_MODEL_DIR") or "."
    model_path = os.path.join(model_dir, "GBM_model_python_1749296476765_1.zip")

    print("Loading MOJO model from:", model_path)

    try:
        # Load the H2O MOJO model
        model = h2o.import_mojo(model_path)
        print("Model loaded successfully.")
    except Exception as e:
        logging.error(f"Failed to load H2O MOJO model: {e}")
        raise RuntimeError("Model loading failed.")


def run(data):
    h2o.init()

# Provide your local path where the MOJO zip is stored (simulate AZUREML_MODEL_DIR)
    model_dir = os.getenv("AZUREML_MODEL_DIR") or "."
    model_path = os.path.join(model_dir, "GBM_model_python_1749296476765_1.zip")
    model = h2o.import_mojo(model_path)

    print("Loading MOJO model from:", model_path)
    

    h2o_test_data = h2o.H2OFrame(data)
    predictions = model.predict(h2o_test_data)
    return predictions

In [46]:
init()
sample_input ={
  "data": [
    {
      "sepal_length": 6.7,
      "sepal_width": 3.1,
      "petal_length": 4.7,
      "petal_width": 1.5
    }
  ]
}

# Run inference
result = run(sample_input)

# Print result
print(result)

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Attempting to start a local H2O server...
  Java Version: openjdk version "23.0.2" 2025-01-21; OpenJDK Runtime Environment Homebrew (build 23.0.2); OpenJDK 64-Bit Server VM Homebrew (build 23.0.2, mixed mode, sharing)
  Starting server from /Users/mdshafiuddin/miniconda3/envs/h2o-mojo-env-v1/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/dg/qkmn114n5zz8qhyh_ywt8cph0000gp/T/tmp2017go65
  JVM stdout: /var/folders/dg/qkmn114n5zz8qhyh_ywt8cph0000gp/T/tmp2017go65/h2o_mdshafiuddin_started_from_python.out
  JVM stderr: /var/folders/dg/qkmn114n5zz8qhyh_ywt8cph0000gp/T/tmp2017go65/h2o_mdshafiuddin_started_from_python.err
  Server is running at http://127.0.0.1:54331
Connecting to H2O server at http://127.0.0.1:54331 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,"2 years, 3 months and 30 days"
H2O_cluster_name:,H2O_from_python_mdshafiuddin_xynj9d
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4.500 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Loading MOJO model from: ./GBM_model_python_1749296476765_1.zip
generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%
Model loaded successfully.
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
[{"predict":"versicolor","setosa":0.001944083,"versicolor":0.9898267126,"virginica":0.0082292044}]


In [56]:
from azure.ai.ml.entities import Environment
env = Environment(
    name="my-h2o-env-test",
    description="Environment with H2O, Azure ML SDK, etc.",
    conda_file="conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04"  # base image
)

ml_client.environments.create_or_update(env)



Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'my-h2o-env-test', 'description': 'Environment with H2O, Azure ML SDK, etc.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/6fab868f-e7ff-4000-b366-b5d0a27cc581/resourceGroups/iat/providers/Microsoft.MachineLearningServices/workspaces/shafi2/environments/my-h2o-env-test/versions/1', 'Resource__source_path': '', 'base_path': '/Users/mdshafiuddin/Desktop/testiat', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x1598fd580>, 'serialize': <msrest.serialization.Serializer object at 0x15990adc0>, 'version': '1', 'conda_file': {'channels': ['conda-forge'], 'dependencies': ['python=3.9', 'openjdk=11.0.15', 'pip', 'requests', 'pyyaml', 'pandas', 'typing_ext

In [59]:
import uuid

# Creating a unique name for the endpoint
online_endpoint_name = "iat-shafi-" + str(uuid.uuid4())[:8]

In [61]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "credit_defaults",
        "model_type": "sklearn.GradientBoostingClassifier",
    },
)

endpoint_result = ml_client.begin_create_or_update(endpoint).result()

print(
    f"Endpint {endpoint_result.name} provisioning state: {endpoint_result.provisioning_state}"
)

Endpint iat-shafi-f2eeefdd provisioning state: Succeeded


In [62]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpint "iat-shafi-f2eeefdd" with provisioning state "Succeeded" is retrieved


In [63]:
from azure.ai.ml.entities import Model


model = ml_client.models.get(name="iat-shafi", version="1")

In [65]:

from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    Environment,
    CodeConfiguration
)
code_config = CodeConfiguration(
    code=".",              # Folder containing score.py
    scoring_script="score.py"  # Your scoring script
)

# Step 3: Attach environment and code config to deployment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment="my-h2o-env-test:1",
    code_configuration=code_config,
    instance_type="Standard_E2s_v3",
    instance_count=1,
)

In [66]:
# Create the deployment
blue_deployment_results = ml_client.online_deployments.begin_create_or_update(
    blue_deployment
).result()

print(f"Deployment state: {blue_deployment_results.provisioning_state}")

Check: endpoint iat-shafi-f2eeefdd exists
Uploading testiat (5.19 MBs): 100%|██████████| 5193753/5193753 [00:01<00:00, 3915685.29it/s]




............................................................................................................Deployment state: Succeeded


In [67]:
deploy_dir = "./deploy"
os.makedirs(deploy_dir, exist_ok=True)

In [68]:
%%writefile {deploy_dir}/sample-request.json
{
  "data": [
    {
      "sepal_length": 6.7,
      "sepal_width": 3.1,
      "petal_length": 4.7,
      "petal_width": 1.5
    }
  ]
}

Writing ./deploy/sample-request.json


In [69]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./deploy/sample-request.json",
    deployment_name="blue",
)

'"[{\\"predict\\":\\"versicolor\\",\\"setosa\\":0.001944083,\\"versicolor\\":0.9898267126,\\"virginica\\":0.0082292044}]"'